# 03 - Tidy Data (wide ↔ long) con `melt`, `pivot` y `pivot_table`

Objetivos prácticos:
- Comprender la filosofía de tidy data: variables en columnas, observaciones en filas.
- Pasar de wide→long con `melt` y de long→wide con `pivot`.
- Resolver duplicados y agregar con `pivot_table` (sum/mean, `margins`, `fill_value`).
- Manipular `MultiIndex` con `stack`/`unstack`, y alternativa `wide_to_long`.

Flujo sugerido:
1) Entender el dataset wide inicial.
2) Derretir (`melt`) a formato long y validar.
3) Volver a wide con `pivot` (y gestionar duplicados con `pivot_table`).
4) Profundizar: `stack/unstack` y `wide_to_long`.

In [4]:
import pandas as pd

print("Construimos un DataFrame wide sencillo:")
w = pd.DataFrame({
    "id": [1, 2, 3],
    "ene": [10, 20, 30],
    "feb": [11, 21, 31],
    "mar": [12, 22, 32],
})
print(w)


Construimos un DataFrame wide sencillo:
   id  ene  feb  mar
0   1   10   11   12
1   2   20   21   22
2   3   30   31   32


## `melt`: de wide a long (paso a paso)

Qué hace:
- Colapsa varias columnas de valores en dos columnas: una con el nombre de la variable y otra con su valor.

Argumentos clave:
- `id_vars`: identificadores que se mantienen fijos (no se derriten).
- `value_vars`: columnas a derretir (si se omite, derrite todas excepto `id_vars`).
- `var_name`/`value_name`: nombres de las nuevas columnas.
- `ignore_index=True`: reindexa el resultado (útil tras melts grandes).
- Columnas jerárquicas (`col_level`) se pueden derretir por nivel específico.

Buenas prácticas:
- Define explícitamente `id_vars` y `value_vars` para evitar sorpresas.
- Si hay meses o categorías, considera ordenar con categorías ordenadas para reportes.

In [5]:
print("Derretimos (melt) columnas de meses -> columnas 'mes' y 'ventas':")
long = pd.melt(
    w,
    id_vars=["id"],
    value_vars=["ene", "feb", "mar"],
    var_name="mes",
    value_name="ventas",
)
print(long)

print("\nOrdenamos 'mes' como categoría ordenada para reportes:")
orden_meses = ["ene", "feb", "mar"]
long["mes"] = pd.Categorical(long["mes"], categories=orden_meses, ordered=True)
print(long.sort_values(["id", "mes"]).head())

print("\nEjemplo con 'ignore_index=True' (reindexar tras melt):")
long_ii = pd.melt(w, id_vars=["id"], value_vars=["ene", "feb", "mar"],
                  var_name="mes", value_name="ventas", ignore_index=True)
print(long_ii.head())


Derretimos (melt) columnas de meses -> columnas 'mes' y 'ventas':
   id  mes  ventas
0   1  ene      10
1   2  ene      20
2   3  ene      30
3   1  feb      11
4   2  feb      21
5   3  feb      31
6   1  mar      12
7   2  mar      22
8   3  mar      32

Ordenamos 'mes' como categoría ordenada para reportes:
   id  mes  ventas
0   1  ene      10
3   1  feb      11
6   1  mar      12
1   2  ene      20
4   2  feb      21

Ejemplo con 'ignore_index=True' (reindexar tras melt):
   id  mes  ventas
0   1  ene      10
1   2  ene      20
2   3  ene      30
3   1  feb      11
4   2  feb      21


## `pivot` vs `pivot_table`: de long a wide y manejo de duplicados

- `pivot(index, columns, values)`: reestructura sin agregación; requiere combinaciones únicas de `index`×`columns`.
- Si hay duplicados, `pivot` lanza `ValueError`. Para eso usa `pivot_table`:
  - `pivot_table(index, columns, values, aggfunc)`: agrega (p. ej. `sum`, `mean`, `count`).
  - `fill_value` para rellenar faltantes y `margins=True` para totales.
- Columnas jerárquicas: varios `values` o agregaciones nombradas pueden crear `MultiIndex` en columnas.
- `stack`/`unstack`: mueven niveles entre filas y columnas; útil para reportes y tidy back.

In [9]:
print("Volver a wide con pivot (requiere unicidad id×mes):")
wide = long.pivot(index="id", columns="mes", values="ventas")
print(wide)

print("\nForzamos duplicados para mostrar el error de pivot:")
long2 = pd.concat([long, long.iloc[[0]]], ignore_index=True)
print("Fila duplicada añadida (mismo id×mes):")
print(long2.head())
print("\nIntento de pivot con duplicados (debe fallar):")
try:
    print(long2.pivot(index="id", columns="mes", values="ventas"))
except Exception as e:
    print("Error esperado:", type(e).__name__, "-", e)




Volver a wide con pivot (requiere unicidad id×mes):
mes  ene  feb  mar
id                
1     10   11   12
2     20   21   22
3     30   31   32

Forzamos duplicados para mostrar el error de pivot:
Fila duplicada añadida (mismo id×mes):
   id  mes  ventas
0   1  ene      10
1   2  ene      20
2   3  ene      30
3   1  feb      11
4   2  feb      21

Intento de pivot con duplicados (debe fallar):
Error esperado: ValueError - Index contains duplicate entries, cannot reshape


In [10]:
print("\nUsando pivot_table para agregar duplicados (sum y mean):")
pvt_sum = long2.pivot_table(index="id", columns="mes", values="ventas", aggfunc="sum", fill_value=0, margins=True)
print("Suma con fill_value=0 y totales (margins=True):")
print(pvt_sum)

pvt_mean = long2.pivot_table(index="id", columns="mes", values="ventas", aggfunc="mean")
print("\nPromedio por id×mes:")
print(pvt_mean)

print("\nstack/unstack para reacomodar niveles:")
stacked = pvt_sum.drop(index="All", errors="ignore").stack()  # vuelve a long
print(stacked.head())
print("\nVolvemos a wide con unstack:")
print(stacked.unstack())

print("\nReset de índice para exportar/tabular fácilmente:")
print(stacked.reset_index(name="ventas"))


Usando pivot_table para agregar duplicados (sum y mean):
Suma con fill_value=0 y totales (margins=True):
mes  ene  feb  mar  All
id                     
1     20   11   12   43
2     20   21   22   63
3     30   31   32   93
All   70   63   66  199

Promedio por id×mes:
mes   ene   feb   mar
id                   
1    10.0  11.0  12.0
2    20.0  21.0  22.0
3    30.0  31.0  32.0

stack/unstack para reacomodar niveles:
id  mes
1   ene    20
    feb    11
    mar    12
    All    43
2   ene    20
dtype: int64

Volvemos a wide con unstack:
mes  ene  feb  mar  All
id                     
1     20   11   12   43
2     20   21   22   63
3     30   31   32   93

Reset de índice para exportar/tabular fácilmente:
    id  mes  ventas
0    1  ene      20
1    1  feb      11
2    1  mar      12
3    1  All      43
4    2  ene      20
5    2  feb      21
6    2  mar      22
7    2  All      63
8    3  ene      30
9    3  feb      31
10   3  mar      32
11   3  All      93


/tmp/ipykernel_245517/931516067.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pvt_sum = long2.pivot_table(index="id", columns="mes", values="ventas", aggfunc="sum", fill_value=0, margins=True)
/tmp/ipykernel_245517/931516067.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pvt_mean = long2.pivot_table(index="id", columns="mes", values="ventas", aggfunc="mean")


## Alternativa: `wide_to_long` (cuando los nombres de columnas codifican variables)

- Útil cuando tienes columnas con patrón como `ene_ventas`, `feb_ventas`, `ene_costos`, ...
- Paramétrico: `stubnames` (raíces), `i` (id), `j` (nueva variable), `sep` y `suffix`.
- Mantiene tidy sin escribir listas largas de `value_vars`.


In [7]:
print("Construimos un wide con ventas y costos por mes:")
w2 = pd.DataFrame({
    "id": [1, 2, 3],
    "ene_ventas": [10, 20, 30], "feb_ventas": [11, 21, 31],
    "ene_costos": [6, 10, 15],  "feb_costos": [7, 11, 16],
})
print(w2)

print("\nUsamos wide_to_long para extraer 'mes' y separar 'ventas' y 'costos':")
wtl = pd.wide_to_long(
    w2,
    stubnames=["ventas", "costos"],
    i="id",
    j="mes",
    sep="_",
    suffix="\\w+",
).reset_index()
print(wtl.sort_values(["id", "mes"]))


Construimos un wide con ventas y costos por mes:
   id  ene_ventas  feb_ventas  ene_costos  feb_costos
0   1          10          11           6           7
1   2          20          21          10          11
2   3          30          31          15          16

Usamos wide_to_long para extraer 'mes' y separar 'ventas' y 'costos':
Empty DataFrame
Columns: [id, mes, ene_ventas, feb_ventas, ene_costos, feb_costos, ventas, costos]
Index: []


## `stack`/`unstack` y `MultiIndex`

- `stack()`: lleva columnas a un nivel de filas (longitudinaliza).
- `unstack(nivel)`: lleva un nivel del índice de filas a columnas.
- Cuando tienes múltiples medidas (p. ej., sum y mean), verás `MultiIndex` en columnas; puedes aplanar o seleccionar con `loc`/`xs`.


In [12]:
print("Creamos tabla agregada con múltiples funciones para ver MultiIndex de columnas:")
ag = (long
      .groupby(["id", "mes"])  # ya es long
      .agg(ventas_sum=("ventas", "sum"), ventas_mean=("ventas", "mean"))
      .reset_index())
print(ag.head())

print("\nPivot con múltiples values crea columnas jerárquicas:")
wide_multi = ag.pivot(index="id", columns="mes", values=["ventas_sum", "ventas_mean"])  # MultiIndex en columnas
print(wide_multi)




Creamos tabla agregada con múltiples funciones para ver MultiIndex de columnas:
   id  mes  ventas_sum  ventas_mean
0   1  ene          10         10.0
1   1  feb          11         11.0
2   1  mar          12         12.0
3   2  ene          20         20.0
4   2  feb          21         21.0

Pivot con múltiples values crea columnas jerárquicas:
    ventas_sum             ventas_mean            
mes        ene   feb   mar         ene   feb   mar
id                                                
1         10.0  11.0  12.0        10.0  11.0  12.0
2         20.0  21.0  22.0        20.0  21.0  22.0
3         30.0  31.0  32.0        30.0  31.0  32.0


/tmp/ipykernel_245517/1816581728.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["id", "mes"])  # ya es long


In [11]:
print("\nSeleccionar un nivel de columnas (solo ventas_sum):")
print(wide_multi["ventas_sum"])  # el sub-bloque

print("\nstack/unstack para mover niveles:")
stacked_multi = wide_multi.stack(0)  # mueve el nivel de columnas externo a filas
print(stacked_multi.head())
print("\nAplanar columnas jerárquicas si se desea exportar fácilmente:")
wide_flat = wide_multi.copy()
wide_flat.columns = [f"{a}__{b}" for a, b in wide_flat.columns.to_flat_index()]
print(wide_flat.reset_index().head())



Seleccionar un nivel de columnas (solo ventas_sum):
mes   ene   feb   mar
id                   
1    10.0  11.0  12.0
2    20.0  21.0  22.0
3    30.0  31.0  32.0

stack/unstack para mover niveles:
mes              ene   feb   mar
id                              
1  ventas_mean  10.0  11.0  12.0
   ventas_sum   10.0  11.0  12.0
2  ventas_mean  20.0  21.0  22.0
   ventas_sum   20.0  21.0  22.0
3  ventas_mean  30.0  31.0  32.0

Aplanar columnas jerárquicas si se desea exportar fácilmente:
   id  ventas_sum__ene  ventas_sum__feb  ventas_sum__mar  ventas_mean__ene  \
0   1             10.0             11.0             12.0              10.0   
1   2             20.0             21.0             22.0              20.0   
2   3             30.0             31.0             32.0              30.0   

   ventas_mean__feb  ventas_mean__mar  
0              11.0              12.0  
1              21.0              22.0  
2              31.0              32.0  


/tmp/ipykernel_245517/2869785535.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stacked_multi = wide_multi.stack(0)  # mueve el nivel de columnas externo a filas
